<a href="https://colab.research.google.com/github/johnson7788/mytest/blob/master/CNN_cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import os
import tensorflow_datasets as tfds
from tensorflow.keras.datasets import cifar10

In [0]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [0]:
x_test, x_vali, y_test, y_vali = x_test[5000:,:], x_test[5000:,:], y_test[5000:,:], y_test[5000:,:],

In [0]:
def one_hot_encode(y, numclass=10):
    """ 
    y是y标签，未做one-hot的标签
    numclass是要做成one-hot的类别
    """
    targets = np.array(y).reshape(-1)
    one_hot_targets = np.eye(numclass)[targets]
    return one_hot_targets

In [0]:
def zscore(x):
    """
    标准化数据
    """
    x_bar = np.mean(x, 0)
    std = np.std(x, 0).clip(min=1)
    return (x - x_bar) / std

In [0]:
x_train, x_test, x_vali = zscore(x_train), zscore(x_test), zscore(x_vali)

In [0]:
y_train = one_hot_encode(y_train)
y_test = one_hot_encode(y_test)
y_vali = one_hot_encode(y_vali)

In [0]:
y_train, y_test
#cifar 10 是物品分类， 图片长和宽是32*32=1024， RGB 三通道， 10分类

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        ...,
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.]]), array([[0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 1., 0., 0.]]))

In [0]:
x_train.shape, y_train.shape

((50000, 32, 32, 3), (50000, 10))

In [0]:
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    """
    输入X，和Y，返回batchsize的生成器
    """
    assert len(inputs) == len(targets)
    # shuffle is used in train the data
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

In [0]:
#构建模型，图的构建部分
def create_model(input_x):
    # 定义一个网络结构：input->conv->relu->pooling->conv->relu->pooling->FC->relu->FC->output
    with tf.name_scope('input'):
        #图片是黑白的，NHWC, C是3,H和W是32，32， N是-1，表示自动推断Batch_size
        picture = tf.reshape(input_x, shape=[-1,32,32,3]) 
        print('输入图片形状')
        print(picture.get_shape())
    
    #卷积层1
    with tf.variable_scope('conv1'):
        # 卷积核格式[H,W,C_IN,C_OUT], H,W是卷积核高度和宽度，C_IN是输入的通道数，20是输出的通道数,也就是
        #卷积核的个数，也就是可以理解为关注的特征数
        filter = tf.get_variable(name='w',shape=[3,3,3,10]) 
        # 每个卷积核都有一个bias，20个卷积核共有20个bias
        bias = tf.get_variable(name='b',shape=[10])
        # strides 步长为1，1， 自动padding，保持形状
        net = tf.nn.conv2d(input=picture, filter=filter, strides=[1,1,1,1], padding='SAME')
        net = tf.nn.bias_add(value=net,bias=bias)    #做完卷积后加上一个bias
        print('卷积层1输出的形状')
        print(net.get_shape())
    # Batch Nomalization
    with tf.variable_scope("BN1"):
        net = tf.layers.batch_normalization(net, training=True, momentum=0.9)
    #卷积层1激活
    with tf.variable_scope('relu1'):
        net = tf.nn.relu(net)
        print('卷积层1激活后输入的形状，激活不改变形状')
        print(net.get_shape())
    #卷积层1池化
    # with tf.variable_scope('pooling1'):
    #     net = tf.nn.max_pool(value=net, ksize=[1,2,2,1], strides=[1,2,2,1],padding='SAME')
    #     print('卷积层1池化后的形状')
    #     print(net.get_shape())
    #卷积层2
    with tf.variable_scope('conv2'):
        # 卷积核格式[H,W,C_IN,C_OUT], H,W是卷积核高度和宽度，C_IN是输入的通道数,是20，30是输出的通道数,也就是
        #卷积核的个数，也就是可以理解为关注的特征数
        filter = tf.get_variable(name='w',shape=[3,3,10,20]) 
        # 每个卷积核都有一个bias，30个卷积核共有30个bias
        bias = tf.get_variable(name='b',shape=[20])
        # 注意输入input是上一个池化层的输出,复制时注意
        # strides 步长为1，1， 自动padding，保持形状
        net = tf.nn.conv2d(input=net, filter=filter, strides=[1,1,1,1], padding='SAME')
        net = tf.nn.bias_add(value=net,bias=bias)    #做完卷积后加上一个bias
        print('卷积层2输出的形状')
        print(net.get_shape())
    # Batch Nomalization
    with tf.variable_scope("BN2"):
        net = tf.layers.batch_normalization(net, training=True, momentum=0.9)
    #卷积层2激活
    with tf.variable_scope('relu2'):
        net = tf.nn.relu(net)
        print('卷积层2激活后输入的形状，激活不改变形状')
        print(net.get_shape())
    #卷积层1池化
    # with tf.variable_scope('pooling2'):
    #     net = tf.nn.max_pool(value=net, ksize=[1,2,2,1], strides=[1,2,2,1],padding='SAME')
    #     print('卷积层2池化后的形状')
    #     print(net.get_shape())
    #做第一个全连接层
    with tf.variable_scope('FC1'):
        shape = net.get_shape()
        #做全连接层，需要把数据拉长1维的，注意维度0是样本数量，即batch_size，不要拉长
        #就是做BP网络
        dim_size = shape[1]*shape[2]*shape[3]   #获取特征数
        net = tf.reshape(net,[-1,dim_size])   #相当于BP网络中的x
        w = tf.get_variable(name='w',shape=[dim_size,500])
        b = tf.get_variable(name='b',shape=[500])
        net = tf.matmul(net,w) + b
        net = tf.nn.relu(net)
        net = tf.nn.dropout(net, keep_prob=0.5)
    #做第二个全连接层
    with tf.variable_scope('FC2'):
        #已知FC1输出的样本维度数是(None, 500), None表示样本的batch_size数量
        #要输出的维度是10，因为预测y是10类，0-9是10个类别,这一层不用激活了,
        # 输出值就是类似sklearn 中的predict_proba, 预测样本属于哪一类的概率
        w = tf.get_variable(name='w',shape=[500,10])
        b = tf.get_variable(name='b',shape=[10])
        logits = tf.matmul(net,w) + b
    return logits

In [0]:
#计算损失
def create_loss(input_y,logits):
    """
    用softmax交叉熵损失函数
    """
    with tf.name_scope('loss'):
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=input_y,logits=logits))
    return loss

In [0]:
#优化模型参数
def train_op(loss,learning_rate=0.05,global_step=None):
    """
    loss: 损失函数
    golbal_step:  训练多少个epoch了
    """
    with tf.name_scope('train_op'):
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)     #做正则时需要更新
        train_op = optimizer.minimize(loss)
        train_oper = tf.group([train_op, update_ops])      #做正则时需要更新
    return train_oper

In [0]:
def create_accuracy(input_y,logits):
    with tf.name_scope("accuracy"):
        y_ture = tf.argmax(input_y,axis=1)
        y_pred = tf.argmax(logits,axis=1)
        accuracy = tf.reduce_mean(tf.cast(tf.equal(y_ture,y_pred),dtype=tf.float32))
    return accuracy

In [0]:
#训练阶段
def train():
    with tf.Graph().as_default():
        input_x = tf.placeholder(dtype=tf.float32,shape=[None,32,32,3], name='input_x')
        input_y = tf.placeholder(dtype=tf.float32,shape=[None,10], name='input_y')
        global_step = tf.train.get_or_create_global_step()  #初始化epoch
        
        #模型网络结构构建
        logits = create_model(input_x)
        #损失函数,返回损失值
        loss = create_loss(input_y,logits)
        #优化损失
        train_oper = train_op(loss=loss,learning_rate=0.01, global_step=global_step)
        #评估模型
        accuracy = create_accuracy(input_y, logits)
        with tf.Session() as sess:
            saver = tf.train.Saver()
            #创建一个持久化对象， 如果使用默认Saver，将所有的模型参数全部持久化
            # 因为不是所有的参数都需要，所以我们仅仅保存可训练模型参数
            save_path = './models/model.ckpt'
            dirpath = os.path.dirname(save_path)
            if not os.path.exists(dirpath):
                os.makedirs(dirpath)
            #如果模型文件已经存在，则尝试恢复模型继续训练
            ckpt = tf.train.get_checkpoint_state(dirpath)
            if ckpt and ckpt.model_checkpoint_path:
                print('模型恢复中。。。')
                saver.restore(sess=sess,save_path=save_path)
            else:
                #否则，初始化所有参数，开始训练
                sess.run(tf.global_variables_initializer())
            
            #设置样本的batch_size
            batch_size = 500
            epoches = 100
            #开始训练模型，直到达到指定条件退出
            for epoch in range(epoches):
                for inputs, targets in iterate_minibatches(x_train, y_train, batch_size, shuffle=True):
                    train_oper_,loss_,accuracy_ = sess.run([train_oper,loss,accuracy],
                                                           feed_dict={input_x:inputs,input_y:targets})
                    print("第%s epoch, 训练集损失%s，准确度%s"%(epoch, loss_, accuracy_*100))
                if epoch%5 == 0:
                    #每5个epoch保存模型1次
                    saver.save(sess=sess, save_path=save_path)
                
                
                #设置验证集,
                for inputs, targets in iterate_minibatches(x_vali, y_vali, batch_size, shuffle=True):
                    valitrain_op_, vali_loss_, vali_accuracy_ = sess.run([train_oper,loss,accuracy], 
                                                                         feed_dict={input_x:inputs,input_y:targets})
                    print("The %s epoch, Validation loss %s，Accuracy %s"%(epoch, vali_loss_, vali_accuracy_*100))
                    #通过判断准确率来停止模型
                    # 训练集准确度和验证集准确度均大于90%时，保存模型，停止训练
                if accuracy_>0.9 and vali_accuracy_>0.9:
                    #退出之前，保存一下模型参数
                    saver.save(sess=sess,save_path=save_path)
                    break


In [0]:
# ! rm -rf models
train()
# x_train.shape

输入图片形状
(?, 32, 32, 3)
卷积层1输出的形状
(?, 32, 32, 10)
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
Instructions for updating:
Please use `layer.__call__` method instead.
卷积层1激活后输入的形状，激活不改变形状
(?, 32, 32, 10)
卷积层2输出的形状
(?, 32, 32, 20)
卷积层2激活后输入的形状，激活不改变形状
(?, 32, 32, 20)
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

第0 epoch, 训练集损失3.0691664，准确度10.599999874830246
第0 epoch, 训练集损失2.9127393，准确度10.000000149011612
第0 epoch, 训练集损失2.6895669，准确度11.59999966621399
第0 epoch, 训练集损失2.524213，准确度13.79999965429306
第0 epoch, 训练集损失2.4221432，准确度18.400000035762787
第0 epoch, 训练集损失2.3

In [0]:
#测试, 注意修改shape, 测试集样本
def prediction():
    with tf.Graph().as_default():
        input_x = tf.placeholder(dtype=tf.float32,shape=[None,32,32,3], name='input_x')
        input_y = tf.placeholder(dtype=tf.float32,shape=[None,10], name='input_y')
        #模型网络结构构建
        logits = create_model(input_x)
        y_predict_proba = tf.nn.softmax(logits)
        #损失函数,返回损失值
        loss = create_loss(input_y,logits)
        #评估模型
        accuracy = create_accuracy(input_y, logits)
        with tf.Session() as sess:
            saver = tf.train.Saver()
            saver.restore(sess=sess, save_path='./models/model.ckpt')
            y_predict_proba_, loss_, accurary_ = sess.run([y_predict_proba, loss, accuracy], feed_dict={input_x: x_test, input_y: y_test})
            print(loss_, accurary_*100)
            print(y_predict_proba_)

In [0]:
prediction()

输入图片形状
(?, 32, 32, 3)
卷积层1输出的形状
(?, 32, 32, 10)
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
Instructions for updating:
Please use `layer.__call__` method instead.
卷积层1激活后输入的形状，激活不改变形状
(?, 32, 32, 10)
卷积层2输出的形状
(?, 32, 32, 20)
卷积层2激活后输入的形状，激活不改变形状
(?, 32, 32, 20)
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

INFO:tensorflow:Restoring parameters from ./models/model.ckpt
0.27686095 91.25999808311462
[[3.60823549e-07 2.95061060e-08 2.57582455e-10 ... 9.99981165e-01
  2.99994390e-10 1.09329875e-07]
 [1.32371059e-09 1.06685629e-08 1.86685583e-07 ... 1.0141587

In [0]:
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers
from keras.callbacks import LearningRateScheduler
import numpy as np
 
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 75:
        lrate = 0.0005
    if epoch > 100:
        lrate = 0.0003
    return lrate
 
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
 
#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)
 
num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)
 
weight_decay = 1e-4
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
 
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))
 
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))
 
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))
 
model.summary()
 
#data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
datagen.fit(x_train)
 
#training
batch_size = 64
 
opt_rms = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])
keras.callbacks.ModelCheckpoint('keras', monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),\
                    steps_per_epoch=x_train.shape[0] // batch_size,epochs=125,\
                    verbose=1,validation_data=(x_test,y_test),callbacks=[LearningRateScheduler(lr_schedule)])
#save to disk
model_json = model.to_json()
with open('model.json', 'w') as json_file:
    json_file.write(model_json)
model.save_weights('model.h5') 
#testing
scores = model.evaluate(x_test, y_test, batch_size=128, verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))

Using TensorFlow backend.












Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-18-922ee824fc84>", line 83, in <module>
    model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),                    steps_per_epoch=x_train.shape[0] // batch_size,epochs=125,                    verbose=1,validation_data=(x_test,y_test),callbacks=[LearningRateScheduler(lr_schedule)])
  File "/usr/local/lib/python3.6/dist-packages/keras/legacy/interfaces.py", line 91, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/keras/engine/training.py", line 1658, in fit_generator
    initial_epoch=initial_epoch)
  File "/usr/local/lib/python3.6/dist-packages/keras/engine/training_generator.py", line 181, in fit_generator
    generator_output = next(output_generator)
  File "/usr/local/lib/python3.6/dist-packages/keras/uti

KeyboardInterrupt: ignored